<a href="https://colab.research.google.com/github/NahuelRepetto/Programacion-Concurrente/blob/main/TrabajoN1Parte2Java.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
%%writefile Matrices.java
import java.io.IOException;
import java.util.ArrayList;

public class Matriz
{  
  //DEFINICION DE CONSTANTES
  private static final int CANT_MAX_ARGS = 1;
  private static final int MAX_DIMENSION = 20;
  private static final int MIN_DIMENSION = 5;
  private static final int MAX_VALOR = 32;
  private static final int MIN_VALOR = -32;
  private static final int MAX_LENGTH_VALUE = 5;

  public static class Hilo extends Thread
  {    
    private int fila;
    private int[][] matrizA;
    private int[][] matrizB;
    private int[][] matrizC;
    
    public Hilo(int fila, int[][] matrizA, int[][] matrizB, int[][] matrizC)
    {
      super();      
      this.fila = fila;
      this.matrizA = matrizA;
      this.matrizB = matrizB;
      this.matrizC = matrizC;
    }

    public void run()
    {      
      for(int i=0; i<matrizA.length; i++)
        for(int j=0; j<matrizB.length; j++)
          matrizC[fila][i] += matrizA[fila][j] * matrizB[j][i];      
    }
  }

  public static int[][] inicializarMatriz(int dimension, int min, int max)
  {
    int [][] matriz = new int [dimension][dimension];
    for(int i=0; i<dimension; i++)
    {
      for(int j=0; j<dimension; j++)          
      matriz[i][j] = (int)(Math.random()*(max-min+1)+min);       
    } 
    return matriz;
  }

  public static void calcularProducto(int fila, int[][] matrizA, int[][] matrizB, int[][] matrizC)
  {
    for(int i=0; i<matrizA.length; i++)
      for(int j=0; j<matrizB.length; j++)
        matrizC[fila][i] += matrizA[fila][j] * matrizB[j][i];  
  }

  public static void mostrarMatriz(int[][] matriz, String nombre)
  {       
    System.out.println("\nMatriz " + nombre);
    for(int i = 0; i < matriz.length; i++) 
    {
      System.out.print("| ");
      for (int j = 0; j < matriz.length; j++)      
        System.out.printf("%" + MAX_LENGTH_VALUE + "d ", matriz[i][j]);              
      System.out.print("  |\n");      
    }  
  }

  public static void compararMatricesCuadradas(int dimension, int[][] matrizA, int[][] matrizB)
  {    
    for(int i=0; i<dimension; i++)
      for(int j=0; j<dimension; j++)
      {
        if(matrizA[i][j] != matrizB[i][j]) 
        {
          System.out.printf("\nLas matrices no son iguales, se encontro una diferencia en la posicion [%d][%d]", i,j);               
          return;
        }       
      }  
    System.out.printf("\nLas matrices son iguales !!!");     
  }

  public static void main(String[] args) throws InterruptedException 
  {    
    //VERIFICO QUE SE HAYA ENVIADO UN ARGUMENTO PARA EL NUMERO DE FILAS/COLUMNAS
    if( args.length != CANT_MAX_ARGS ) 
    {
      System.out.println("Debe ingresar SOLO el numero de filas/columnas, dentro del rango de " + MIN_DIMENSION + " a " + MAX_DIMENSION); 
      System.exit(1);
    }    

    //VALIDO QUE EL ARGUMANTO PASADO SEA VALIDO  
    try
    {
      Integer.parseInt(args[0]); 
    }
    catch (NumberFormatException nfe)
    {
      System.out.println("Tipo de argumento inválido.");
      System.exit(1);
    }
   
    int n = Integer.parseInt(args[0]); //DIMENSIN DE LA MATRIZ  

    //VERIFICO QUE SE HAYA ENVIADO EL NUMERO DENTRO DE LOS LIMITES ESTABLECIDOS
    if( n < MIN_DIMENSION || n > MAX_DIMENSION ) 
    {
     System.out.println("El numero de filas/columnas debe estar dentro del rango de " + MIN_DIMENSION + " a " + MAX_DIMENSION);    
     System.exit(1);
    }  
    
    //INICIALIZO LAS MATRICES, A Y B CON VALORES RANDOM, CS Y CH EN 0
    int [][] matrizA = inicializarMatriz(n, MIN_VALOR, MAX_VALOR);
    int [][] matrizB = inicializarMatriz(n, MIN_VALOR, MAX_VALOR);
    int [][] matrizCS = inicializarMatriz(n, 0, 0);
    int [][] matrizCH = inicializarMatriz(n, 0, 0);
    
    //CALCULO VALORES DE CS (SECUENCIAL)
    for(int i=0; i<n; i++)    
      calcularProducto(i, matrizA, matrizB, matrizCS);
    
    //CALCULO VALORES DE CH (CON CONCURRENCIA)
    //CREO UNA ESTRUCTURA PARA ALMACENAR LAS REFERENCIAS A LOS HILOS CREADOS    
    ArrayList<Hilo> hilos = new ArrayList<Hilo>();

    //CREO N HILOS
    for(int i=0; i<n; i++)  
      hilos.add(new Hilo(i, matrizA, matrizB, matrizCH));     
   
    //INICIO A LOS N HILOS 
    for (Hilo h: hilos) 
      h.start();          
           
    //ESPERO A LOS N HILOS 
    for (Hilo h: hilos) 
      h.join();
    
    //MUESTRO LOS RESULTADOS
    mostrarMatriz(matrizA,"A");    
    mostrarMatriz(matrizB,"B");
    mostrarMatriz(matrizCS,"CS"); 
    mostrarMatriz(matrizCH,"CH");

    //COMPARO CS Y CH    
    compararMatricesCuadradas(n, matrizCS, matrizCH);
  }
}

Overwriting Matrices.java


In [8]:
!java Matrices.java 5


Matriz A
|     7    14    19    18     3   |
|    26     8    -3    10   -25   |
|   -26   -21   -18     5   -13   |
|    32    12     9    -4   -28   |
|   -13   -12    -1   -11     4   |

Matriz B
|   -14    -4    31     9    32   |
|     7   -31   -23    22    11   |
|     7    31    18   -30   -12   |
|    -3    24    19     3    16   |
|   -10     2   -19   -31    -5   |

Matriz CS
|    49   565   522  -238   423   |
|  -109  -255  1233  1305  1241   |
|   206   291  -305   262  -702   |
|    -9  -373  1334  1138  1124   |
|    84   137  -430  -508  -732   |

Matriz CH
|    49   565   522  -238   423   |
|  -109  -255  1233  1305  1241   |
|   206   291  -305   262  -702   |
|    -9  -373  1334  1138  1124   |
|    84   137  -430  -508  -732   |

Las matrices son iguales !!!